In [ ]:
pip install scgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.3 MB/s eta 0:00:00


In [ ]:
from scgraph.geographs.marnet import marnet_geograph
import folium

In [ ]:
import geopandas as gpd
import json
from shapely.geometry import Point

f = open('ports.json')

# returns JSON object as
# a dictionary
data = json.load(f)
ports_gdf = gpd.GeoDataFrame(data)

# Step 3: Convert latitude and longitude to Point geometry
ports_gdf['geometry'] = ports_gdf.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)

start_coordinates = " 81.64182088655531,21.216768604756734"

your_point = gpd.GeoDataFrame({'geometry': [Point(start_coordinates.split(',')[0], start_coordinates.split(',')[1])]}, crs=ports_gdf.crs)
nearest_port_index = ports_gdf.distance(your_point.unary_union)
sorted = nearest_port_index.sort_values()
indexes = sorted.iloc[:3].index
print(indexes)
import requests
from pyproj import CRS, Transformer
crs_hk80 = CRS.from_proj4("+proj=tmerc +lat_0=22.31213333333334 +lon_0=114.1785555555556 +k=1 +x_0=836694.05 +y_0=819069.8 +ellps=intl +towgs84=-162.619,-276.959,-161.764,0.067753,-2.24365,-1.15883,-1.09425 +units=m +no_defs ")
transformer = Transformer.from_crs(crs_hk80, 4326)
transformer.transform(835000, 812000)
INTERSECTION_ID = 'Start'
url = "https://api.openrouteservice.org/v2/directions/driving-car"
# api_key = "5b3ce3597851110001cf6248846d6a68ec334aaaa34195cdcd3f11ad"
api_key = "5b3ce3597851110001cf6248bef52906ad594343afbb98f64d4f4fa0"
failed_coords = []
# end_coordinates = "72.963540, 18.952151"
m = folium.Map(location=(start_coordinates.split(',')[1], start_coordinates.split(',')[0]))
folium.Marker((start_coordinates.split(',')[1], start_coordinates.split(',')[0]), tooltip=folium.Tooltip(INTERSECTION_ID, permanent=True)).add_to(m)


ports = []
for i in indexes:
  ports.append((ports_gdf.iloc[i]["Latitude"],ports_gdf.iloc[i]["Longitude"]))
  print(ports_gdf.iloc[i]["Latitude"],ports_gdf.iloc[i]["Longitude"])
  end_coordinates = str(ports_gdf.iloc[i]["Longitude"]) + "," + str(ports_gdf.iloc[i]["Latitude"])
  params = {
      "api_key": api_key,
      "start": start_coordinates,
      "end": end_coordinates
  }

  response = requests.get(url, params=params)
  COORDINATES_LIST = []
  if response.status_code == 200:
      data = response.json()
      COORDINATES_LIST = data["features"][0]["geometry"]["coordinates"]
      COORDINATES_LIST = [[coord[1], coord[0]] for coord in COORDINATES_LIST]
      folium.PolyLine(COORDINATES_LIST, color='red').add_to(m)
  else:
    failed_coords.append(end_coordinates)
    print("Failed to fetch directions. Status code:", response.status_code)




<ipython-input-19-b51f5bf6323e>:13: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  ports_gdf['geometry'] = ports_gdf.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)


Index([1295, 1291, 1268], dtype='int64')
19.301944444 84.966666667
Failed to fetch directions. Status code: 404
17.683333 83.3
Failed to fetch directions. Status code: 404
17.633333333 83.25
Failed to fetch directions. Status code: 404


In [ ]:

o = ports[0][0]
d = ports[0][1]
eo = 13.965731860806589
ed = 109.1672395852882

ox = ports[1][0]
dx = ports[1][1]
eox = 13.965731860806589
edx = 109.1672395852882


oy = ports[2][0]
dy = ports[2][1]
eoy = 13.965731860806589
edy = 109.1672395852882


In [ ]:
output = marnet_geograph.get_shortest_path(
    origin_node={"latitude":o,"longitude":d},
    destination_node={"latitude": eo,"longitude": ed},
     output_units='km'
)

output1 = marnet_geograph.get_shortest_path(
    origin_node={"latitude": ox,"longitude":dx },
    destination_node={"latitude":eox,"longitude": edx},
     output_units='km'
)

output2 = marnet_geograph.get_shortest_path(
    origin_node={"latitude": oy, "longitude":dy},
    destination_node={"latitude":eoy,"longitude":edy},
     output_units='km'
)



In [ ]:
x = (([[i['latitude'],i['longitude']] for i in output['coordinate_path']]))
y = (([[i['latitude'],i['longitude']] for i in output1['coordinate_path']]))
z = (([[i['latitude'],i['longitude']] for i in output2['coordinate_path']]))

# Show the length
lengthx = int(output['length']) #=> Length:  19596.4653
lengthy = int(output1['length']) #=> Length:  19596.4653
lengthz = int(output2['length']) #=> Length:  19596.4653

print(lengthx)
print(lengthy)
print(lengthz)


find_shortest_route = [lengthx,lengthy,lengthz]

shortest_route = (min(find_shortest_route))

4796
4573
4576


In [ ]:
line_colorx = "blue"
line_colory ="blue"
line_colorz = "blue"

for i in range(len(find_shortest_route)):
  if find_shortest_route[i] == shortest_route:
    if i == 0:
      print("Green x")
      line_colorx= "green"

    if i == 1:
      print("Green y")
      line_colorx= "green"

    if i == 2:
      print("Green z")
      line_colorx= "green"


Green y


In [ ]:
folium.PolyLine(x, color=line_colorx, weight=2.5, opacity=1 ,tooltip=('km',lengthx)).add_to(m)
folium.PolyLine(y, color=line_colory, weight=2.5, opacity=1,tooltip=('km',lengthy)).add_to(m)
folium.PolyLine(z, color=line_colorz, weight=2.5, opacity=1,tooltip=('km',lengthz)).add_to(m)

In [ ]:
m

In [ ]:
# Inputs
distance_km = length
weight_kg = 0
fuel_consumption_km = 10  # Assume 10 liters/km for demonstration purposes
fuel_price_per_liter = 2  # Assume $2 per liter for demonstration purposes
operating_costs = 500     # Assume $500 per voyage for demonstration purposes

# Calculate fuel usage for the given distance and weight
fuel_usage_liters = fuel_consumption_km * distance_km

# Calculate fuel cost
fuel_cost = fuel_usage_liters * fuel_price_per_liter

# Calculate total cost including fuel and operating costs
total_cost = fuel_cost + operating_costs

print(f"Estimated cost for a {distance_km} km sea route with {weight_kg} kg weight: ₹{total_cost}")


NameError: name 'length' is not defined

In [ ]:
!pip install geographiclib

In [ ]:
import requests

# Define the API endpoint
url = "https://api.openrouteservice.org/v2/directions/driving-hgv"

# Provide your API key
api_key = "5b3ce3597851110001cf6248846d6a68ec334aaaa34195cdcd3f11ad"

# Define the start and end coordinates
start_coordinates = "72.640066,23.061456"
end_coordinates = "69.68641269890959, 21.65165445974833"

# Define the parameters
params = {
    "api_key": api_key,
    "start": start_coordinates,
    "end": end_coordinates
}

# Send the request
response = requests.get(url, params=params)
COORDINATES_LIST = []
# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    # Access the directions data
    COORDINATES_LIST = data["features"][0]["geometry"]["coordinates"]

else:
    print("Failed to fetch directions. Status code:", response.status_code)

In [ ]:
COORDINATES_LIST = [[coord[1], coord[0]] for coord in COORDINATES_LIST]



In [ ]:
COORDINATES_LIST

[[23.061514, 72.639986],
 [23.060759, 72.639335],
 [23.060393, 72.639027],
 [23.059185, 72.638014],
 [23.058643, 72.637414],
 [23.058388, 72.637175],
 [23.05814, 72.636847],
 [23.05797, 72.636631],
 [23.057479, 72.635971],
 [23.05723, 72.635605],
 [23.057095, 72.635405],
 [23.056737, 72.634881],
 [23.056183, 72.633998],
 [23.055884, 72.633523],
 [23.05578, 72.633369],
 [23.055317, 72.63268],
 [23.054905, 72.632066],
 [23.054026, 72.63066],
 [23.053761, 72.630287],
 [23.053731, 72.630229],
 [23.051768, 72.627162],
 [23.051049, 72.626092],
 [23.050089, 72.624622],
 [23.049561, 72.623811],
 [23.049234, 72.62331],
 [23.048876, 72.622767],
 [23.048593, 72.622328],
 [23.048002, 72.621421],
 [23.047512, 72.620658],
 [23.047042, 72.619921],
 [23.046916, 72.619725],
 [23.046565, 72.619174],
 [23.045858, 72.618153],
 [23.043125, 72.614402],
 [23.040913, 72.611393],
 [23.040713, 72.61114],
 [23.039641, 72.609667],
 [23.038883, 72.608675],
 [23.038428, 72.608206],
 [23.037933, 72.607811],
 [23.037

In [ ]:
from pyproj import CRS, Transformer
crs_hk80 = CRS.from_proj4("+proj=tmerc +lat_0=22.31213333333334 +lon_0=114.1785555555556 +k=1 +x_0=836694.05 +y_0=819069.8 +ellps=intl +towgs84=-162.619,-276.959,-161.764,0.067753,-2.24365,-1.15883,-1.09425 +units=m +no_defs ")
transformer = Transformer.from_crs(crs_hk80, 4326)
transformer.transform(835000, 812000)
COORDINATES = (22.246757209968482, 114.16457352372427)
INTERSECTION_ID = 'idk'
import folium
# Instantiate map with centre COORDINATES
m = folium.Map(location=COORDINATES)
# add a marker to map (m) in COORDINATES, which it has permanent tooltip showing INTERSECTION_ID
folium.Marker(COORDINATES, tooltip=folium.Tooltip(INTERSECTION_ID, permanent=True)).add_to(m)
# Draw a red line in map (m) on COORDINATES_LIST, on click it will shows STREET_ENAME
folium.PolyLine(COORDINATES_LIST, color='red').add_to(m)

In [ ]:
m